In [35]:
# #https://python.langchain.com/docs/integrations/document_loaders/mediawikidump/

# from langchain_community.document_loaders import MWDumpLoader
# loader = MWDumpLoader(
#     file_path="stardewvalleywiki.com_mediawiki-20240505-wikidump\stardewvalleywiki.com_mediawiki-20240505-current.xml",
#     encoding="utf-8",
#     skip_redirects=True,
#     stop_on_error=True,
# )

# #does not export the fricking price 
# # documents=[doc.page_content for doc in loader.load()]

# documents=loader.load()


In [1]:
# #https://github.com/earwig/mwparserfromhell
# import mwxml
# import mwparserfromhell
# from langchain_core.documents import Document
# from tqdm import tqdm



# stardew_wiki_dump = mwxml.Dump.from_file(open("stardewvalleywiki.com_mediawiki-20240505-wikidump\stardewvalleywiki.com_mediawiki-20240505-current.xml", encoding="utf-8"))

# documents=[]


# def load_single_page_from_dump(page):
#     for revision in page:
#         code = mwparserfromhell.parse(revision.text)
#         text= code.strip_code(
#             normalize=True, collapse=True, keep_template_params=True
#         )
#         metadata = {"source":page.title}
#         text=text.split('Category')[0]
#         return Document(page_content=text, metadata=metadata)

# for page in tqdm(stardew_wiki_dump.pages):
#     if not page.redirect:
#         documents.append(load_single_page_from_dump(page))
        


572it [00:02, 193.33it/s]


KeyboardInterrupt: 

In [14]:
#https://github.com/earwig/mwparserfromhell
import mwxml
import mwparserfromhell
from langchain_core.documents import Document
from tqdm import tqdm



stardew_wiki_dump = mwxml.Dump.from_file(open("stardewvalleywiki.com_mediawiki-20240505-wikidump\stardewvalleywiki.com_mediawiki-20240505-current.xml", encoding="utf-8"))

raw_documents=[]
    

for page in tqdm(stardew_wiki_dump.pages):
    if not page.redirect:
        for revision in page:
            # if page.title =="Abigail":
            code = mwparserfromhell.parse(revision.text)
            # print(f'================Code=================\n{code}\n================Code=================')
            # text= code.strip_code(
            #     normalize=False, collapse=True, keep_template_params=True
            # )
            # print(f'================Text=================\n{text}\n================Text=================')
            metadata = {"source":page.title}
            code=code.split('[[Category')[0]
            
            raw_documents.append(Document(page_content=code, metadata=metadata))
                # break
                
# print(documents[0].page_content)

2829it [00:14, 190.74it/s]


In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import Any

splitter = RecursiveCharacterTextSplitter(
    add_start_index=True,
    chunk_size=1000,
    is_separator_regex=True,
    # separators = [r"\w(=){3}\n", r"\w(=){2}\n", r"\n\n", r"\n"],
    separators = [r"\n\n", r"\n"],
    # keep_seperator=False
)

# documents=splitter.create_documents(documents)
documents=splitter.split_documents(raw_documents)

In [29]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(
    model_name="all-MiniLM-L6-v2")

d:\User\Documents - Data Drive\Kyle\Projects\Misc\StardewGPT\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\User\Documents - Data Drive\Kyle\Projects\Misc\StardewGPT\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\User\Documents - Data Drive\Kyle\Projects\Misc\StardewGPT\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [30]:
from langchain_chroma import Chroma 
#8minutes
vector_db = Chroma.from_documents(documents, embedding_function)#,persist_directory='db')

In [2]:
#Load from disk 
from langchain_chroma import Chroma
vector_db=Chroma(persist_directory='db', embedding_function=embedding_function)

In [60]:
#test retriever
retriever = vector_db.as_retriever()

query="When is Mayor Lewis' birthday?"
retriever.get_relevant_documents(query)

[Document(page_content='In parallel with his work as mayor, he is also involved in the Stardew Valley Agricultural Fund. He may send the player a letter that includes money from the fund "to help you continue your good work", such as using it to buy seeds.\n\nHe has never been married and has no family; all he lives for is being mayor. He sometimes questions if it was worth it.\n\nLewis is particularly good friends with [[Marnie]], as shown in [[Quests#Mayor\'s &ldquo;Shorts&rdquo;|the "Mayor\'s Shorts"]] story quest. He can also sometimes be found inside [[Marnie\'s Ranch]] saying, "I\'m just here to make sure all of Marnie\'s business needs are met." Additionally, his and Marnie\'s 6-heart event confirms that they are in a relationship, although he does not want the relationship to become public knowledge.\n\n==Gifts==\n{{GiftHeader}}\n\n===Love===\n{{Quote|Wow, this is my favorite! Thank you!}}\n\'\'Stardrop Tea\'\'\n{{Quote|I\'ve sampled many teas in my life, but none are quite so 

In [33]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Ollama

llm = Ollama(model="llama3")


prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [61]:
rag_chain.invoke("Where can you find Mayor Lewis' shorts?")

"You can find Mayor Lewis' shorts in Marnie's bedroom."

In [62]:
rag_chain.invoke("When is Mayor Lewis' birthday?")

"According to the context, Mayor Lewis' birthday is in Spring on the 7th day."

In [4]:
# Test if the model could understand MediaWiki tables 
from langchain_community.llms import Ollama

llm = Ollama(model="llama3")

prompt = """
Given the context below, answer the following question:
Context:
{{Infobox villager
|portrait  = Abigail.png
|birthday  = {{Season|Fall}} 13
|location  = Pelican Town
|address   = [[Pierre's General Store]]
|family    = {{NPC|Pierre|Father}}{{NPC|Caroline|Mother}}
|friends   = {{NPC|Sam}}{{NPC|Sebastian}}
|marriage  = Yes
|clinic    = {{Season|Spring}} 4
|favorites = {{name|Amethyst}}{{name|Banana Pudding}}{{name|Blackberry Cobbler}}{{name|Chocolate Cake}}{{name|Pufferfish}}{{name|Pumpkin}}{{name|Spicy Eel}}
}}
Question: When is Abigail's birthday and who are her parents?
"""

llm.invoke(prompt)

"According to the context, Abigail's birthday is on Fall 13. Her parents are Pierre (father) and Caroline (mother)."

In [3]:
#TODO:
#1. Investigate optimal chunk size for the splitter - Try bigger chunk sizes?, also fix the crop header issue 
#2. Optimizie the retriever, find a better retriever
#3. Have LLMs evals (ragas)
#4. Implement in Chainlit 